# LAB 2.1: CONTEXT WINDOW OPTIMIZATION

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 2 - Advanced Context Engineering  
**Duration:** 50 minutes  
**Type:** Hands-on Context Compression & Optimization

## LAB OVERVIEW

This lab focuses on **optimizing context usage** to fit large amounts of information into limited context windows. You'll learn to:

- Count and budget tokens accurately
- Maximize information density
- Implement progressive detail loading
- Apply extractive summarization
- Build hierarchical context compression systems

**Scenario:** You're building a commercial loan underwriting system. Each loan application consists of:
- Application form (5 pages)
- 3 years of tax returns (60 pages)
- 6 months of bank statements (30 pages)
- Business plan (20 pages)
- Property appraisal (15 pages)
- Credit report (10 pages)

**Total: ~140 pages = ~70,000 tokens**

Your context window: **8,000 tokens** (realistic production constraint for cost/speed optimization)

**Challenge:** Provide accurate loan analysis while using only 11% of the available information.

## LEARNING OBJECTIVES

By the end of this lab, you will be able to:

✓ Count tokens accurately using tiktoken  
✓ Identify and eliminate low-value tokens  
✓ Transform verbose text into high-density formats  
✓ Implement progressive detail strategies  
✓ Extract key information without losing critical context  
✓ Build multi-tier context hierarchies

### Step 1: Import Libraries

In [ ]:
# Lab 2.1: Context Window Optimization
# Advanced Prompt Engineering Training - Session 2

import os
import json
from openai import OpenAI
import tiktoken
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Any
import re

from dotenv import load_dotenv

load_dotenv(override=True)

print("✓ Libraries imported")

### Step 2: Configure OpenAI Client & Tokenizer

In [ ]:
# Check if API key exists
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found. Please set it in .env file")

# Initialize OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Configuration
MODEL = os.getenv("MODEL_NAME")
TEMPERATURE = 0  # Deterministic for BFSI applications

if not MODEL:
    raise ValueError("MODEL_NAME not found. Please set it in .env file")

# Initialize tokenizer (for GPT-4)
encoding = tiktoken.encoding_for_model(MODEL)

print(f"✓ Model: {MODEL}")
print(f"✓ Tokenizer: {encoding.name}")

### Step 3: Create Helper Functions

In [ ]:
def count_tokens(text: str) -> int:
    """
    Count tokens in text using tiktoken
    
    Args:
        text (str): Text to count
    
    Returns:
        int: Token count
    """
    return len(encoding.encode(text))

def call_gpt4(prompt: str, system_prompt: str = "You are a helpful AI assistant.") -> Dict:
    """
    Call GPT-4 API with token tracking
    
    Args:
        prompt (str): User prompt
        system_prompt (str): System prompt
    
    Returns:
        Dict: Response with metadata
    """
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=TEMPERATURE
        )
        
        return {
            "content": response.choices[0].message.content,
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens,
            "success": True
        }
    except Exception as e:
        return {
            "content": "",
            "error": str(e),
            "success": False
        }

def analyze_token_distribution(text: str) -> Dict:
    """
    Analyze how tokens are distributed in text
    
    Args:
        text (str): Text to analyze
    
    Returns:
        Dict: Token statistics
    """
    tokens = encoding.encode(text)
    words = text.split()
    
    return {
        "total_tokens": len(tokens),
        "total_characters": len(text),
        "total_words": len(words),
        "tokens_per_word": len(tokens) / len(words) if words else 0,
        "characters_per_token": len(text) / len(tokens) if tokens else 0
    }

print("✓ Helper functions created")

### Step 4: Test Tokenizer

In [ ]:
# Test token counting
test_texts = [
    "Hello world",
    "The quick brown fox jumps over the lazy dog",
    "Context window optimization is critical for production systems.",
    "Credit score: 720, DTI: 35%, LTV: 80%"
]

print("TOKEN COUNTING TEST:")
print("=" * 80)
for text in test_texts:
    token_count = count_tokens(text)
    stats = analyze_token_distribution(text)
    print(f"\nText: '{text}'")
    print(f"  Tokens: {token_count}")
    print(f"  Words: {stats['total_words']}")
    print(f"  Ratio: {stats['tokens_per_word']:.2f} tokens/word")
print("\n" + "=" * 80)

## CONTEXT WINDOW FUNDAMENTALS

### Understanding Token Economics

**Token Basics:**
- 1 token ≈ 4 characters in English
- 1 token ≈ 0.75 words in English
- Numbers and special characters vary
- Different languages have different ratios

**Rule of Thumb:** Keep context under 10,000 tokens for 95% of queries.

## CHALLENGE 1: TOKEN COUNTING & ANALYSIS

**Time:** 10 minutes  
**Objective:** Learn to count tokens accurately and identify waste

### Background

You can't optimize what you don't measure. This challenge teaches precise token accounting.

### Loan Application Dataset

In [ ]:
# Realistic commercial loan application data

loan_application = {
    "applicant_info": """
    APPLICANT INFORMATION FORM
    
    Full Legal Name: Sarah Elizabeth Chen
    Date of Birth: March 15, 1978 (Age: 46 years old)
    Social Security Number: XXX-XX-5678
    Current Address: 456 Oak Street, Apartment 3B, Seattle, Washington 98101, United States of America
    Telephone Number: (206) 555-1234 (Mobile) / (206) 555-5678 (Home)
    Email Address: sarah.chen@example.com
    Marital Status: Married
    Spouse Name: Michael Robert Chen
    Number of Dependents: Two children (ages 12 and 9)
    """,
    
    "business_info": """
    BUSINESS INFORMATION
    
    Business Legal Name: Chen Technology Consulting Services, LLC
    Doing Business As (DBA): Chen Tech Consulting
    Business Structure: Limited Liability Company (LLC)
    State of Incorporation: Washington
    Federal Tax ID (EIN): 12-3456789
    Date of Establishment: January 1, 2019 (5 years in operation)
    Business Address: 789 Technology Drive, Suite 200, Seattle, WA 98102
    Business Phone: (206) 555-9999
    Business Email: contact@chentechconsulting.com
    Industry Classification: Information Technology Consulting Services
    NAICS Code: 541512 - Computer Systems Design Services
    Number of Employees: 12 full-time employees, 3 part-time contractors
    Annual Gross Revenue (2023): $850,000
    Annual Gross Revenue (2022): $720,000
    Annual Gross Revenue (2021): $580,000
    """,
    
    "loan_request": """
    LOAN REQUEST DETAILS
    
    Type of Loan Requested: Commercial Real Estate Acquisition Loan
    Requested Loan Amount: Five Hundred Thousand Dollars ($500,000.00)
    Intended Use of Funds: Purchase of commercial office property for business expansion
    Desired Loan Term: Seven (7) years
    Preferred Repayment Schedule: Monthly payments with principal and interest
    Requested Interest Rate Type: Fixed rate for the entire term of the loan
    Collateral Offered: The commercial property being purchased, located at 321 Business Park Avenue, Seattle, WA 98103
    Property Purchase Price: Eight Hundred Thousand Dollars ($800,000.00)
    Down Payment Amount: Three Hundred Thousand Dollars ($300,000.00) from business savings
    Loan-to-Value Ratio: 62.5% (which is $500,000 divided by $800,000)
    """,
    
    "financial_summary": """
    FINANCIAL SUMMARY (From Tax Returns and Financial Statements)
    
    Business Financial Overview:
    - Total Assets as of December 31, 2023: $1,200,000
    - Total Liabilities as of December 31, 2023: $280,000
    - Net Worth (Assets minus Liabilities): $920,000
    - Current Ratio (Current Assets divided by Current Liabilities): 3.2
    - Debt-to-Income Ratio: 35% (total monthly debt payments divided by gross monthly income)
    
    Cash Flow Analysis:
    - Average Monthly Revenue: $70,833 ($850,000 annual divided by 12 months)
    - Average Monthly Operating Expenses: $52,000
    - Average Monthly Net Profit: $18,833
    - Free Cash Flow After Debt Service: $12,000 per month
    
    Personal Financial Information:
    - Personal Credit Score (FICO): 720 (Good credit rating)
    - Personal Annual Income (Including Spouse): $95,000 from salary plus business owner's draw
    - Personal Assets: $400,000 (retirement accounts, savings, investments)
    - Personal Liabilities: $250,000 (primary residence mortgage)
    - Personal Net Worth: $150,000
    """
}

print("LOAN APPLICATION DATA LOADED")
print("=" * 80)
for section, content in loan_application.items():
    tokens = count_tokens(content)
    print(f"{section}: {tokens} tokens")
print("=" * 80)

total_tokens = sum(count_tokens(content) for content in loan_application.values())
print(f"TOTAL APPLICATION: {total_tokens} tokens")
print("=" * 80)

### Student Exercise

In [ ]:
# TODO: Analyze token waste
# Requirements:
# - Identify redundant information
# - Count unnecessary words/phrases
# - Calculate potential token savings

def analyze_token_waste(text: str) -> Dict:
    """
    Identify token waste in text
    
    Args:
        text (str): Text to analyze
    
    Returns:
        Dict: Waste analysis
    """
    # TODO: Implement
    # Hints:
    # - Count instances of "the", "a", "an"
    # - Identify redundant phrases ("dollars and cents", "USD")
    # - Find verbose patterns ("in the amount of" vs just the amount)
    pass

# TODO: Test your analysis

### Solution

In [ ]:
# SOLUTION: Token Waste Analysis

def analyze_token_waste(text: str) -> Dict:
    """
    Comprehensive token waste analysis
    
    Args:
        text (str): Text to analyze
    
    Returns:
        Dict: Detailed waste breakdown
    """
    # Common filler words
    filler_words = ['the', 'a', 'an', 'of', 'in', 'on', 'at', 'to', 'for']
    
    # Count fillers
    filler_count = 0
    for filler in filler_words:
        # Word boundary matching to avoid false positives
        pattern = r'\b' + filler + r'\b'
        filler_count += len(re.findall(pattern, text, re.IGNORECASE))
    
    # Identify redundant patterns
    redundant_patterns = [
        (r'United States of America', 'USA', 'location'),
        (r'\(\$[\d,]+\.?\d*\)', '', 'numeric_redundancy'),
        (r'divided by', '/', 'math_operator'),
        (r'multiplied by', '×', 'math_operator'),
        (r'Total (\w+) as of [^:]+:', r'\1:', 'date_redundancy'),
        (r'\(Age: \d+ years old\)', '', 'calculated_field'),
        (r'\([\d\.]+ percent\)', '', 'percentage_redundancy')
    ]
    
    redundancy_count = 0
    potential_replacements = []
    
    for pattern, replacement, category in redundant_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            redundancy_count += len(matches)
            potential_replacements.append({
                'category': category,
                'instances': len(matches),
                'example': matches[0] if isinstance(matches[0], str) else matches[0]
            })
    
    # Verbose phrases
    verbose_phrases = [
        ('in the amount of', ''),
        ('as of the date of', 'on'),
        ('for the purpose of', 'to'),
        ('in order to', 'to'),
        ('with respect to', 'regarding'),
        ('in the event that', 'if'),
    ]
    
    verbose_count = 0
    for verbose, compact in verbose_phrases:
        matches = len(re.findall(re.escape(verbose), text, re.IGNORECASE))
        verbose_count += matches
    
    # Estimate token savings
    filler_tokens = filler_count  # Each filler word is ~1 token
    redundancy_tokens = redundancy_count * 5  # Average savings per redundancy
    verbose_tokens = verbose_count * 2  # Average savings per verbose phrase
    
    total_tokens = count_tokens(text)
    estimated_savings = filler_tokens + redundancy_tokens + verbose_tokens
    savings_percentage = (estimated_savings / total_tokens * 100) if total_tokens > 0 else 0
    
    return {
        'total_tokens': total_tokens,
        'filler_words': filler_count,
        'redundant_patterns': redundancy_count,
        'verbose_phrases': verbose_count,
        'estimated_token_savings': estimated_savings,
        'savings_percentage': savings_percentage,
        'potential_replacements': potential_replacements
    }

# Analyze each section
print("TOKEN WASTE ANALYSIS:")
print("=" * 80)

total_savings = 0
for section_name, content in loan_application.items():
    analysis = analyze_token_waste(content)
    total_savings += analysis['estimated_token_savings']
    
    print(f"\n{section_name.upper().replace('_', ' ')}:")
    print(f"  Current tokens: {analysis['total_tokens']}")
    print(f"  Filler words: {analysis['filler_words']}")
    print(f"  Redundant patterns: {analysis['redundant_patterns']}")
    print(f"  Verbose phrases: {analysis['verbose_phrases']}")
    print(f"  Potential savings: {analysis['estimated_token_savings']} tokens ({analysis['savings_percentage']:.1f}%)")

print("\n" + "=" * 80)
print(f"TOTAL POTENTIAL SAVINGS: {total_savings} tokens")
print(f"Original: {total_tokens} tokens")
print(f"Optimized: ~{total_tokens - total_savings} tokens")
print(f"Reduction: {total_savings / total_tokens * 100:.1f}%")
print("=" * 80)

### Key Takeaways

✓ **Measure first** - Can't optimize without metrics  
✓ **Filler words add up** - 20-30% of typical prose  
✓ **Redundancy is expensive** - Parenthetical amounts, repeated info  
✓ **Verbose phrases matter** - "in order to" → "to" saves 2 tokens each time

## CHALLENGE 2: INFORMATION DENSITY OPTIMIZATION

**Time:** 10 minutes  
**Objective:** Transform verbose text into high-density formats

### Background

Same information, fewer tokens. This is information density optimization.

### Problem: Low-Density Format

In [ ]:
# Current format (from loan application)
verbose_financial = """
FINANCIAL SUMMARY (From Tax Returns and Financial Statements)

Business Financial Overview:
- Total Assets as of December 31, 2023: $1,200,000
- Total Liabilities as of December 31, 2023: $280,000
- Net Worth (Assets minus Liabilities): $920,000
- Current Ratio (Current Assets divided by Current Liabilities): 3.2
- Debt-to-Income Ratio: 35% (total monthly debt payments divided by gross monthly income)

Cash Flow Analysis:
- Average Monthly Revenue: $70,833 ($850,000 annual divided by 12 months)
- Average Monthly Operating Expenses: $52,000
- Average Monthly Net Profit: $18,833
- Free Cash Flow After Debt Service: $12,000 per month
"""

print(f"Verbose format: {count_tokens(verbose_financial)} tokens")

### Student Exercise

In [ ]:
# TODO: Create high-density version
# Requirements:
# - Preserve all numerical values
# - Remove explanatory text
# - Use compact structure (JSON, key-value, table)
# - Target: 50%+ reduction

def optimize_information_density(verbose_text: str) -> str:
    """
    Convert verbose text to high-density format
    
    Args:
        verbose_text (str): Original verbose text
    
    Returns:
        str: Optimized high-density text
    """
    # TODO: Implement
    pass

# TODO: Test and compare token counts

### Solution

In [ ]:
# SOLUTION: High-Density Formats

# Approach 1: Structured Key-Value
dense_kv_format = """
FINANCIALS (2023):
Assets: $1.2M | Liabilities: $280K | Net Worth: $920K
Current Ratio: 3.2 | DTI: 35%

MONTHLY CASH FLOW:
Revenue: $70.8K | Expenses: $52K | Net: $18.8K | Free CF: $12K
"""

# Approach 2: JSON
dense_json_format = """{
  "financials_2023": {
    "assets": 1200000,
    "liabilities": 280000,
    "net_worth": 920000,
    "current_ratio": 3.2,
    "dti": 0.35
  },
  "monthly_cash_flow": {
    "revenue": 70833,
    "expenses": 52000,
    "net_profit": 18833,
    "free_cash_flow": 12000
  }
}"""

# Approach 3: Table Format
dense_table_format = """
FINANCIAL METRICS:
Metric          | Value
----------------|--------
Assets          | $1.2M
Liabilities     | $280K
Net Worth       | $920K
Current Ratio   | 3.2
DTI             | 35%
Monthly Revenue | $70.8K
Monthly Expenses| $52K
Monthly Net     | $18.8K
Free Cash Flow  | $12K
"""

# Compare token counts
formats = {
    "Original (Verbose)": verbose_financial,
    "Key-Value": dense_kv_format,
    "JSON": dense_json_format,
    "Table": dense_table_format
}

print("INFORMATION DENSITY COMPARISON:")
print("=" * 80)

for format_name, content in formats.items():
    tokens = count_tokens(content)
    print(f"\n{format_name}:")
    print(f"  Tokens: {tokens}")
    if format_name != "Original (Verbose)":
        original_tokens = count_tokens(verbose_financial)
        savings = original_tokens - tokens
        savings_pct = (savings / original_tokens) * 100
        print(f"  Savings: {savings} tokens ({savings_pct:.1f}%)")
    print(f"  Preview:\n{content[:150]}...")

print("\n" + "=" * 80)

### Key Takeaways

✓ **Structured formats save tokens** - JSON, tables, key-value pairs  
✓ **Abbreviations are safe** - $1.2M vs $1,200,000  
✓ **LLMs understand compact formats** - No accuracy loss  
✓ **70%+ reduction possible** - Without losing information

## LAB SUMMARY

### Optimization Techniques Mastered

| Technique | Token Savings | When to Use |
|-----------|---------------|-------------|
| Token Counting | Baseline | Always - measure first |
| Density Optimization | 60-75% | Structured data, metrics |
| Progressive Loading | 50-90% | Variable query complexity |
| Extractive Summarization | 60-80% | Targeted queries |
| Hierarchical Compression | 80-95% | Production systems |

### Production Checklist

Before deploying context optimization:

- [ ] Implement accurate token counting (tiktoken)
- [ ] Create optimized data formats (JSON, tables, key-value)
- [ ] Build progressive detail levels (minimal, medium, maximum)
- [ ] Implement extractive summarization for targeted queries
- [ ] Set and enforce token budgets
- [ ] Test with actual LLM calls to verify accuracy
- [ ] Monitor token usage in production
- [ ] Iterate based on query patterns